In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

from quasar_drw_v3 import quasar_drw as qso_drw


In [3]:
f = h5py.File("../DES_qso.h5", "r")

In [4]:
ID            = f["ID"][:]
ra            = f["ra"][:]
dec           = f["dec"][:]
redshift      = f["redshift"][:]
flag          = f["flag"][:]

In [5]:
bands = ["g", "r", "i", "z"]

In [6]:
ra_list       = []
dec_list      = []
redshift_list = []

redshift_select = []
i_mean_list     = []
flag_list       = []

In [7]:
qso_count     = 0
ozDES_count   = 0
S82_count     = 0

ozDES_count_all = 0
S82_count_all   = 0

In [8]:
for i, name in enumerate(ID):
    
    z        = redshift[i]
    qso_flag = flag[i]
    
    band_count = 0    
    
    ## how many spectroscopically comfirmed quasars in the data
    if qso_flag == "Q":
        S82_count_all += 1
    elif qso_flag =="QQ":
        ozDES_count_all += 1
    
    
    for j, band in enumerate(bands):
    
        if (name+"/SDSS_"+band+"_mjd" in f) and (name+"/DES_"+band+"_mjd" in f) :
            
            ## make sure this band has SDSS and DES data
            band_count += 1
            
            ## read-in i-band data
            if band == "i":
            
                mjd       = f[name+"/DES_"+band+"_mjd"][:]
                mag       = f[name+"/DES_"+band+"_mag"][:]
                mag_error = f[name+"/DES_"+band+"_mag_err"][:]
                lc  = qso_drw(mjd, mag, mag_error, z, preprocess=True)

                mjd       = f[name+"/SDSS_"+band+"_mjd"][:]
                mag       = f[name+"/SDSS_"+band+"_mag"][:]
                mag_error = f[name+"/SDSS_"+band+"_mag_err"][:]
                lc2 = qso_drw(mjd, mag, mag_error, z, preprocess=True)
                time, signal, error = lc2.get_lc()
        
                # merge 2 light curves
                lc.add_lc(time, signal, error, preprocess=False)
                time, signal, error = lc.get_lc()
    
                i_signal = np.array(signal)
    
    
    ## if at least 2 bands have data
    if band_count >= 2:
        confirm = False
        qso_count += 1
        
        ra_list.append(ra[i])
        dec_list.append(dec[i])
        redshift_list.append(z)
        
        ## check if spectroscopically comfirmed
        if qso_flag == "Q":
            confirm = True
            S82_count += 1
        elif qso_flag == "QQ":
            confirm = True
            ozDES_count +=1
        
        ## save spectroscopically comfirmed quasar data
        if confirm:
            #print(qso_flag)
            redshift_select.append(z)
            i_mean = np.mean(i_signal)
            i_mean_list.append(i_mean)
            flag_list.append(flag[i])
        
        #print(qso_flag)
        
print("total qso count = " + str(qso_count))

/Users/wtliao/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:639: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/wtliao/anaconda3/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:158: RuntimeWarning: invalid value encountered in greater
  _filtered_data.mask |= _filtered_data > max_value
/Users/wtliao/anaconda3/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:159: RuntimeWarning: invalid value encountered in less
  _filtered_data.mask |= _filtered_data < min_value


total qso count = 921


In [9]:
print( "Total Q-flag qso: " + str(S82_count_all))

Total Q-flag qso: 1829


In [10]:
print( "Total QQ-flag qso: " + str(ozDES_count_all))

Total QQ-flag qso: 0


In [11]:
print( "We have select spectroscopically confirmed quasar: " + str(S82_count))

We have select spectroscopically confirmed quasar: 449


In [12]:
ozDES_count

0

#### Statistics for spectroscopically confirmed qso:

In [13]:
redshift_select = np.array(redshift_select)

print( "mean redshift = ", np.mean(redshift_select))
print( "min  redshift = ", np.min(redshift_select) )
print( "max  redshift = ", np.max(redshift_select) )

mean redshift =  1.86448997773
min  redshift =  0.198
max  redshift =  4.77


In [14]:
i_mean_list = np.array(i_mean_list)

print( "mean i-mag = ", np.mean(i_mean_list))
print( "min  i-mag = ", np.min(i_mean_list) )
print( "max  i-mag = ", np.max(i_mean_list) )

mean i-mag =  21.1457013017
min  i-mag =  17.1265034198
max  i-mag =  23.2325418052


In [15]:
quasar_all = np.vstack((ra_list, dec_list, redshift_list))

In [16]:
np.savetxt("quasar_select.txt", quasar_all, fmt="%10.6f")

In [17]:
check_readin = np.loadtxt("quasar_select.txt")

In [18]:
print(check_readin[2, :])

[ 1.029   1.834   0.5     0.5     1.6676  1.786   1.6312  0.5     1.4544
  2.944   2.9     2.1512  2.401   2.75    1.9016  3.5     0.9916  1.6
  0.9916  1.912   1.096   1.829   2.7     0.3364  1.818   2.8688  2.393
  1.614   1.9     0.964   0.3988  2.588   2.6712  0.974   0.9292  2.289
  1.557   1.246   1.712   1.647   1.145   0.82    1.2828  1.711   2.061
  1.331   1.833   1.896   3.5     0.6796  1.5532  1.642   1.8756  1.08
  1.9588  0.404   1.5     1.0488  1.831   1.912   2.11    1.12    2.6608
  1.7196  1.824   1.3556  1.484   2.224   1.8444  1.144   1.14    1.3
  1.503   3.898   1.0124  2.132   1.054   3.8     2.2032  2.687   0.9968
  1.8756  2.842   1.8236  0.7576  0.652   2.484   1.016   3.046   2.0888
  1.246   0.57    0.921   2.668   2.196   1.279   2.4528  2.295   2.555
  0.808   1.679   1.9744  1.774   2.412   1.      2.055   1.829   0.352
  0.5     1.628   0.1     2.624   1.9172  1.977   3.439   1.3452  1.343
  1.9068  0.4     0.431   1.8392  1.9952  2.611   1.4856  0.967  

In [19]:
redshift_list

[1.0289999999999999,
 1.8340000000000001,
 0.5,
 0.5,
 1.6676,
 1.786,
 1.6312,
 0.5,
 1.4543999999999999,
 2.944,
 2.8999999999999999,
 2.1511999999999998,
 2.4009999999999998,
 2.75,
 1.9016,
 3.5,
 0.99160000000000004,
 1.6000000000000001,
 0.99160000000000004,
 1.9119999999999999,
 1.0960000000000001,
 1.829,
 2.7000000000000002,
 0.33639999999999998,
 1.8180000000000001,
 2.8687999999999998,
 2.3929999999999998,
 1.6140000000000001,
 1.8999999999999999,
 0.96399999999999997,
 0.39879999999999999,
 2.5880000000000001,
 2.6711999999999998,
 0.97399999999999998,
 0.92920000000000003,
 2.2890000000000001,
 1.5569999999999999,
 1.246,
 1.712,
 1.647,
 1.145,
 0.81999999999999995,
 1.2827999999999999,
 1.7110000000000001,
 2.0609999999999999,
 1.331,
 1.833,
 1.8959999999999999,
 3.5,
 0.67959999999999998,
 1.5531999999999999,
 1.6419999999999999,
 1.8755999999999999,
 1.0800000000000001,
 1.9588000000000001,
 0.40400000000000003,
 1.5,
 1.0488,
 1.831,
 1.9119999999999999,
 2.109999999